In [1]:
!pip install -q transformers
!pip install -q transformers[torch]
!pip install -q accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch

In [ ]:
from google.colab import drive
import os

def adjust_to_num(dec):
  if dec == 'positive':
    return 2
  elif dec == 'neutral':
    return 1
  else:
    return 0
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/HackTJ 2024/Newskip')
df = pd.read_csv('data.csv')
df['label'] = df['decision'].apply(adjust_to_num)
df = df.drop('decision', axis=1)
df['text'] = df['title']
df = df.drop('title', axis=1)
df = df.drop(df[df['label'] == 1].index)
train = df[:5745]
test = df[5745:]
#train.head()

In [ ]:
text, sentiment = list(train['text']), list(train['label'])
labels = train['label'].tolist()
sentences = train['text'].tolist()

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
inputs = tokenizer(sentences, padding="max_length", truncation=True)

In [ ]:
class NewsDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

train_ds = NewsDataset(inputs, labels)

In [ ]:
print(train_ds.__getitem__(0))

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds
)
trainer.train()

In [ ]:
   torch.save(model.state_dict(), './model_better')

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)
model.load_state_dict(torch.load('./model_better'))
model.eval()

In [ ]:
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct/len(y_pred))*100
  return acc

In [ ]:
predictions = []

for text in test['text']:
  # Tokenize input text
  inputs = tokenizer(text, return_tensors='pt')

  # Perform prediction
  outputs = model(**inputs.to(device))
  predicted_class = torch.argmax(outputs.logits)

  # Interpret prediction
  predictions.append(predicted_class)

accuracy_fn(torch.tensor(predictions),torch.tensor(test['label'].tolist()))

In [ ]:
# Prepare input text
text = ""

# Tokenize input text
inputs = tokenizer(text, return_tensors='pt')

# Perform prediction
outputs = model(**inputs.to(device))
predicted_class = torch.argmax(outputs.logits)

# Interpret prediction
class_names = ['negative', 'neutral', 'positive']
predicted_sentiment = class_names[predicted_class]

print("Predicted sentiment:", predicted_sentiment)

In [ ]:
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/HackTJ 2024/Newskip')

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

def pred(input):
    model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)
    model.load_state_dict(torch.load('./model_better', map_location=torch.device('cpu')))
    model.eval()
    text = input
    tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits)
    class_names = ['negative', 'neutral', 'positive']
    predicted_sentiment = class_names[predicted_class]
    print("Predicted sentiment:", predicted_sentiment)